In [6]:
# Import required packages
import pandas as pd

In [7]:
# Read in data
data = pd.read_csv('/Users/kloeyap/ybell-data-analysis/data/gsheets_clean.csv', delimiter='\t')
data

,Contract Name,Source,Sub Source,Stream Month,Territory,Release Title,Release Artist,ISRC,Track Title,Distribution Channel,Configuration,Units,Net Payable
0,ebb (PLT-SprjsB0s),FUGA,Yandex,October 2022,RU,Bay Area Wave Collection,ebb,US23A1545496,Linda Mar III,Digital,Streaming,2.0,0.001047
1,ebb (PLT-SprjsB0s),FUGA,Yandex,October 2022,RU,Bay Area Wave Collection,ebb,US23A1545351,Kelly's Cove III,Digital,Streaming,3.0,0.001570
2,ebb (PLT-SprjsB0s),FUGA,Yandex,October 2022,RU,Bay Area Wave Collection,ebb,US23A1545495,Linda Mar II,Digital,Streaming,1.0,0.000523
3,ebb (PLT-SprjsB0s),FUGA,Yandex,October 2022,RU,Bay Area Wave Collection,ebb,US23A1545494,Linda Mar I,Digital,Streaming,1.0,0.000523
4,ebb (PLT-SprjsB0s),FUGA,Yandex,October 2022,RU,Bay Area Wave Collection,ebb,US23A1545354,Rockaway Beach III,Digital,Streaming,3.0,0.001570
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186160,KAIYO (PLTM-YI2ybeW4),Apple Music,Apple Music,May 2023,ZA,Palmyra,KAIYO,US23A1560959,Free Diving,Digital,General Stream,3.0,0.004630
186161,KAIYO (PLTM-YI2ybeW4),Apple Music,Apple Music,May 2023,BE,Surf Odyssey,KAIYO,US23A1540826,Whenever at Sea,Digital,General Stream,1.0,0.004451
186162,KAIYO (PLTM-YI2ybeW4),Apple Music,Apple Music,May 2023,LV,Palmyra,KAIYO,US23A1560959,Free Diving,Digital,General Stream,1.0,0.001499
186163,KAIYO (PLTM-YI2ybeW4),Apple Music,Apple Music,May 2023,MG,Changing Tide,KAIYO,US23A1545500,High Tide,Digital,General Stream,2.0,0.001532


In [8]:
data.groupby('Distribution Channel')[['Distribution Channel']].count()

,Distribution Channel
Distribution Channel,
Digital,186156
Licensing,6


In [28]:
sourceinfo = data.groupby('Source')[['Source']].count()
sourceinfo = sourceinfo.rename(columns={'Source': 'Count'})
sourceinfo['proportion'] = sourceinfo['Count'].apply(lambda x : x / sum(sourceinfo['Count']))
sourceinfo.sort_values(by ='proportion', ascending=False)


,Count,proportion
Source,,
Apple Music,164167,0.881836
Spotify,14032,0.075374
FUGA,4923,0.026444
YouTube,1996,0.010722
YouTube Red,775,0.004163
iTunes,162,0.000870
iTunes Match,110,0.000591


As expected, Apple Music leads the way with about 88% of streams coming from the platform, with Spotify following, but with only about 7.5%.

In [30]:
artist_count = data.groupby('Release Artist')[['Release Artist']].count()
artist_count = artist_count.rename(columns = {'Release Artist' : 'count'})
artist_count['proportion'] = artist_count['count'] / sum(artist_count['count'])
artist_count.sort_values(by='proportion', ascending=False)


,count,proportion
Release Artist,,
chamberecho,90498,0.486243
KAIYO,54324,0.291881
Sharks Deli,13873,0.074539
ebb,10738,0.057695
m a e a,5326,0.028616
the spaceheads,4893,0.026290
MAIKU,3694,0.019848
Ric Wavy,2771,0.014888


chamberecho is the most popular artist with 48% of streams coming from them, followed by KAIYO. The two together combine with a total of about 77% of all streams. 

In [19]:
artist_pay = data.groupby('Release Artist')[['Net Payable']].sum().sort_values(by = 'Net Payable', ascending=False)
artist_pay['Most Streamed Source'] = data.groupby('Release Artist')


,Net Payable
Release Artist,
chamberecho,137269.951853
KAIYO,53757.476410
Sharks Deli,7285.406975
ebb,5462.936536
m a e a,3731.992999
the spaceheads,2215.257145
MAIKU,1239.761743
Ric Wavy,1023.064199


In [22]:
data.groupby(['Release Artist', 'Source'])[['Source']].count()

Source
Release Artist Source              
KAIYO          Apple Music    51972
               FUGA            1152
               Spotify          944
               YouTube          157
               YouTube Red       68
               iTunes            26
               iTunes Match       5
MAIKU          Apple Music     2379
               FUGA              86
               Spotify          486
               YouTube          521
               YouTube Red      218
               iTunes Match       4
Ric Wavy       Apple Music     1675
               FUGA             125
               Spotify          588
               YouTube          328
               YouTube Red       44
               iTunes             4
               iTunes Match       7
Sharks Deli    Apple Music    10839
               FUGA             444
               Spotify         1970
               YouTube          412
               YouTube Red      116
               iTunes            39
               iTunes Match      53
chamberecho    Apple Music    79331
               FUGA            2093
               Spotify         8289
               YouTube          405
               YouTube Red      272
               iTunes            86
               iTunes Match      22
ebb            Apple Music     9075
               FUGA             717
               Spotify          830
               YouTube           68
               YouTube Red       43
               iTunes Match       5
m a e a        Apple Music     4967
               FUGA              58
               Spotify          289
               YouTube            4
               iTunes Match       8
the spaceheads Apple Music     3881
               FUGA             248
               Spotify          636
               YouTube          101
               YouTube Red       14
               iTunes             7
               iTunes Match       6

In [12]:
data.groupby('Territory')[['Territory']].count()

,Territory
Territory,
AD,3
AE,1795
AF,1
AG,415
AI,167
...,...
YE,6
YT,1
ZA,3203


In [13]:
data.groupby('Stream Month')[['Stream Month']].count()

,Stream Month
Stream Month,
April 2022,9727
April 2023,8747
August 2022,12979
December 2022,8801
February 2022,9098
February 2023,7862
January 2022,6988
January 2023,7759
July 2022,12007


June 2023 seemed to have the most streams. **The plan is to convert this data into datetime, so it can actually be sorted correctly, then plotted into a line plot to analyze the trends with respect to time. **

## Looking into chamberecho (most streamed artist)

In [18]:
chamberecho = data[data['Release Artist'] == 'chamberecho']
chamberecho

,Contract Name,Source,Sub Source,Stream Month,Territory,Release Title,Release Artist,ISRC,Track Title,Distribution Channel,Configuration,Units,Net Payable
9144,chamberecho,Spotify,Spotify,January 2022,US,lazy Sunday morning,chamberecho,US23A1542153,lazy Sunday morning,Digital,Streaming,1.0,0.001562
9145,chamberecho,Spotify,Spotify,January 2022,US,california night,chamberecho,US23A1543075,california night,Digital,Streaming,1.0,0.001015
9146,chamberecho,Spotify,Spotify,January 2022,AR,monday watch the movie,chamberecho,US23A1541863,monday watch the movie,Digital,Streaming,1.0,0.000523
9147,chamberecho,Spotify,Spotify,January 2022,GB,monday watch the movie,chamberecho,US23A1541863,monday watch the movie,Digital,Streaming,3.0,0.004611
9148,chamberecho,Spotify,Spotify,January 2022,US,cuddle in the sheets,chamberecho,US23A1541864,cuddle in the sheets,Digital,Streaming,1.0,0.001550
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163444,chamberecho (PLTM-zqYv2j8i),Apple Music,Apple Music,June 2023,RO,academic ambiance,chamberecho,US23A1562606,exam euphony,Digital,General Stream,135.0,0.292175
163445,chamberecho (PLTM-zqYv2j8i),Apple Music,Apple Music,June 2023,RO,immersive noise for sleep,chamberecho,US23A1559111,cold brew nation,Digital,General Stream,25.0,0.054106
163446,chamberecho (PLTM-zqYv2j8i),Apple Music,Apple Music,June 2023,TW,international study,chamberecho,US23A1560085,mon amour,Digital,General Stream,1.0,0.002309
163447,chamberecho (PLTM-zqYv2j8i),Apple Music,Apple Music,June 2023,TW,dog days,chamberecho,US23A1551451,snooze fest,Digital,General Stream,1.0,0.001442
